<a href="https://colab.research.google.com/github/Vishy-A/MachineLearningIntro/blob/main/HW6/4105HW6P1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [132]:
#Vishy Adusumilli
#801185663
#HW Problem 1

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from collections import OrderedDict
from google.colab import drive


drive.mount('/content/drive')
file_path = '/content/drive/My Drive/Machine Learning/Datasets/Housing.csv'
dataset = pd.DataFrame(pd.read_csv(file_path))
dataset.head()



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [133]:
varsall = ['area', 'bedrooms', 'bathrooms', 'stories', 'mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'parking', 'prefarea', 'furnishingstatus', 'price']
varstrings = ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea', 'furnishingstatus']

def binarymap(inp):
  return inp.map({'yes' : 1, 'no' : 0, 'furnished' : 2, 'semi-furnished' : 1, 'unfurnished' : 0 })

dataset[varstrings] = dataset[varstrings].apply(binarymap)


dataset.head()


,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,1,0,0,0,1,2,1,2
1,12250000,8960,4,4,4,1,0,0,0,1,3,0,2
2,12250000,9960,3,2,2,1,0,1,0,0,2,1,1
3,12215000,7500,4,2,2,1,0,1,0,1,3,1,2
4,11410000,7420,4,1,2,1,1,1,0,1,2,0,2


In [134]:
standardScaler = StandardScaler()

dataset[varsall] = standardScaler.fit_transform(dataset[varsall])
np.random.seed(0)
dataset.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,4.566365,1.046726,1.403419,1.421812,1.378217,0.405623,-0.465315,-0.734539,-0.219265,1.472618,1.517692,1.804941,1.406286
1,4.004484,1.757010,1.403419,5.405809,2.532024,0.405623,-0.465315,-0.734539,-0.219265,1.472618,2.679409,-0.554035,1.406286
2,4.004484,2.218232,0.047278,1.421812,0.224410,0.405623,-0.465315,1.361397,-0.219265,-0.679063,1.517692,1.804941,0.091662
3,3.985755,1.083624,1.403419,1.421812,0.224410,0.405623,-0.465315,1.361397,-0.219265,1.472618,2.679409,1.804941,1.406286
4,3.554979,1.046726,1.403419,-0.570187,0.224410,0.405623,2.149083,1.361397,-0.219265,1.472618,1.517692,-0.554035,1.406286


In [135]:
x = dataset.values
y = dataset.pop('price')

x = torch.tensor(x)
y = torch.tensor(y)

In [136]:
samples = x.shape[0]
n_val = int(.2 * samples)

shuffledindices = torch.randperm(samples)

trainindices = shuffledindices[:-n_val]
valindices = shuffledindices[-n_val:]

trainindices, valindices

(tensor([501, 475,  63,   3,  33, 486, 317, 223, 146, 319, 536, 321, 234, 159,
         272,  57, 156, 494, 263, 248, 282, 298, 209, 428, 358, 141, 160,  71,
         408,  29, 301, 469, 149, 316,  77, 493,   4, 219, 505, 120, 108, 372,
         393, 389, 174, 430, 238, 452, 216,  44, 353, 392, 329, 476, 247, 331,
         197, 124, 344, 189, 145, 460, 270,  76, 338, 250, 406, 447, 446, 115,
         488, 498, 165,  96,  43, 431, 336, 422,  85, 187,  31, 318, 411, 292,
          22, 171, 181, 166, 214, 113,  36, 522, 379,  39, 135,  82,  12, 407,
         244, 242, 526, 168, 366, 178, 148, 504, 241, 275, 491,  94,  74, 440,
         448, 123, 122,  86, 412, 459, 516, 537, 289,  46, 544, 473,  32,  50,
         251,  83, 380, 449, 417, 253,   9, 307,  18, 386, 474, 200, 364, 210,
           8, 442, 306, 503, 363,  48, 273, 371,   2, 401,  30, 347, 357, 296,
         461, 140, 398, 266, 284, 164,  64,  52, 217, 355, 497, 283, 142, 359,
         237, 143,  54, 541, 109, 162, 268, 119, 370

In [137]:
xtrain = x[trainindices]
ytrain = y[trainindices]

valx = x[valindices]
valy = y[valindices]

xntrain = .1 * xtrain
xnval = .1 * valx

In [138]:
xtrain = x[trainindices]
ytrain = y[trainindices]

valx = x[valindices]
valy = y[valindices]


In [139]:
def training_loop(epochs, optimizer, model, loss_func, xtrain, ytrain, xtest, ytest):
  for epoch in range(1, epochs + 1):
    t_p_train = model(xtrain)
    loss_train = loss_func(t_p_train, ytrain)

    t_p_val = model(xtest)
    vloss = loss_func(t_p_val, ytest)

    optimizer.zero_grad()
    loss_train.backward()
    optimizer.step()


    if epoch % 500 == 0 :
      print(f"Epoch {epoch}, Training Loss {loss_train.item():.4f}, " f" Validation Loss {vloss.item():.4f}")



In [140]:
def loss_func(t_p, t_c):
  squared_diffs = (t_p - t_c) ** 2
  return squared_diffs.mean()

In [141]:
sequential_model = nn.Sequential(OrderedDict([
    ('hidden_linear', nn.Linear(1,32)),
    ('hidden_activation', nn.Tanh()),
    ('output_linear', nn.Linear(32, 1))
]))
sequential_model

Sequential(
  (hidden_linear): Linear(in_features=1, out_features=32, bias=True)
  (hidden_activation): Tanh()
  (output_linear): Linear(in_features=32, out_features=1, bias=True)
)

In [142]:
optimizer = optim.SGD(sequential_model.parameters(), lr = 1e-2)

training_loop(
    epochs = 3000,
    optimizer = optimizer,
    model = sequential_model,
    loss_func = loss_func,
    xtrain = xntrain.to(torch.float32),
    ytrain = ytrain.to(torch.float32),
    xtest = xnval.to(torch.float32),
    ytest = valy.to(torch.float32)
)


RuntimeError: ignored